# Model Drift Alerting Example

Shows how to detect model drift and trigger an alert.

In [1]:
import pandas as pd
from scipy.stats import ks_2samp
import requests
import os

## Load Reference and Production Data

In [2]:
ref = pd.read_csv('../data/processed/infra_training.csv')
prod = pd.read_csv('../data/processed/features.csv')

## Compute KS Statistic for Selected Features

In [3]:
features = [c for c in ref.columns if c not in ['structure_id', 'failure_within_1yr']]
drift = {}
for f in features:
    stat, p = ks_2samp(ref[f], prod[f])
    drift[f] = {'ks_stat': stat, 'p_value': p}
drifted = [f for f, v in drift.items() if v['p_value'] < 0.05]
if drifted:
    print('Drift detected in:', drifted)
    # Example: trigger alert (Slack)
    webhook_url = os.environ.get('SLACK_WEBHOOK_URL')
    if webhook_url:
        requests.post(webhook_url, json={'text': f'DRIFT ALERT: {drifted}'})
else:
    print('No significant drift detected.')